In [ ]:
import pickle
import torch
from scipy.io import wavfile
import argparse
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from model import PedalNet

MODEL = "./hm2.ckpt"
DATA = "data.pickle"

model = PedalNet.load_from_checkpoint(MODEL)
model.eval()
data = pickle.load(open(DATA, "rb"))

x_test = data["x_test"]
y_test = data["y_test"]
print(x_test.dtype)
print(max(abs(x_test.flatten())))
prev_sample = np.concatenate((np.zeros_like(x_test[0:1]), x_test[:-1]), axis=0)
pad_x_test = np.concatenate((prev_sample, x_test), axis=2)


In [ ]:
y_pred = []
for x in np.array_split(pad_x_test, 10):
    y_pred.append(model(torch.from_numpy(x)).detach().numpy())

y_pred = np.concatenate(y_pred)
y_pred = y_pred[:, :, -x_test.shape[2] :]

In [ ]:
print(data["std"])
print(data["mean"])

In [ ]:
y_pred = y_pred.flatten() * 10 * data["std"] + data["mean"]
y_test = y_test.flatten() * 10 * data["std"] + data["mean"]

plt.figure(figsize=(16,9))
plt.title("x_test")
plt.plot(data["x_test"].flatten())
plt.figure(figsize=(16,9))
plt.title("y_pred")
plt.plot(y_pred.flatten())
plt.figure(figsize=(16,9))
plt.title("y_test")
plt.plot(y_test)

In [ ]:
error = abs(y_test) - abs(y_pred)
plt.figure(figsize=(16,9))
plt.plot(error)

In [ ]:
import random
def get_random_window(actual, predicted, window=1000):
    total_size = len(predicted)
    start = int(random.uniform(0, total_size-window))
    end = start + window
    return (actual[start:end], predicted[start:end])


start = 150000
end = start + 2000
plt.figure(figsize=(16,9))
plt.plot(y_test[start:end], label="Actual")
plt.plot(y_pred.flatten()[start:end], label="Predicted")
plt.legend()

for i in range(5):
    plt.figure(figsize=(16,9))
    actual, predicted = get_random_window(y_test.flatten(), y_pred.flatten())
    plt.plot(actual, label="Actual")
    plt.plot(predicted, label="Predicted")
    plt.legend()
    plt.savefig(f"actual_vs_pred_{i}.png")

In [ ]:
for i in range(5):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(16,9))
    actual, predicted = get_random_window(y_test.flatten(), y_pred.flatten(), 10000)
    axes[0].specgram(actual, Fs=44100, mode="psd", detrend="mean")
    axes[1].specgram(predicted, Fs=44100, mode="psd", detrend="mean")
    axes[0].set_title("Actual")
    axes[1].set_title("Predicted")
    fig.suptitle("Spectrogram of random 10k sample window")
    fig.savefig(f"spectrogram_{i}.png")